## 카카오 로컬 API 활용
- 도로명 주소 ---> 위도, 경도 좌표

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key

In [2]:
with open('keys/카카오apikey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [3]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '강원특별자치도 춘천시 중앙로길 135'
url = f'{base_url}?query={quote(addr)}'

In [4]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [5]:
result = requests.get(url, headers=header).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [6]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [7]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.5164625580995, 126.906658750671)

In [8]:
df = pd.read_csv('data/station_list2.csv')
df

,지역명,측정소명,주소,운영기관,설치년도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0
...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0


In [10]:
result['documents']

{'documents': [{'address': {'address_name': '서울 영등포구 영등포동3가 33',
    'b_code': '1156010400',
    'h_code': '1156053500',
    'main_address_no': '33',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_h_name': '영등포동',
    'region_3depth_name': '영등포동3가',
    'sub_address_no': '',
    'x': '126.906658750671',
    'y': '37.5164625580995'},
   'address_name': '서울 영등포구 경인로 지하 843',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 영등포구 경인로 지하 843',
    'building_name': '영등포역지하도상가',
    'main_building_no': '843',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_name': '영등포동3가',
    'road_name': '경인로',
    'sub_building_no': '',
    'underground_yn': 'Y',
    'x': '126.906658750671',
    'y': '37.5164625580995',
    'zone_no': '07305'},
   'x': '126.906658750671',
   'y': '37.5164625580995'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [33]:
# 측정소 좌표 얻는 함수
lat_list, lng_list = [], []         # 위도, 경도 리스트 초기화
for i in range(len(df.주소)):       
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'    
    url = f'{base_url}?query={df.주소[i]}'          # 측정소 주소 데이터 가져와서 좌표 검색
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    

    # 좌표 데이터를 찾을 수 없는 경우를 대비한 예외코드 추가
    try:
        latitude = result['documents'][0]['y']
        longitude = result['documents'][0]['x']
        lat_list.append(latitude)
        lng_list.append(longitude)
    except IndexError:
        lat_list.append(None)
        lng_list.append(None)

df['위도'] = lat_list
df['경도'] = lng_list
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.5179681611717,127.047059839521
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.5450208674213,127.13679935135
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.6479342499954,127.011869707413
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.5446697303679,126.835179100171
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.4873960036367,126.927113892829
...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.4890451973362,126.500458122035
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.4815759715974,126.520156152743
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.5395361411297,126.643719243436
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.4096918414655,126.268544091817


In [34]:
df.to_csv('data/측정소좌표.csv')
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.5179681611717,127.047059839521
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.5450208674213,127.13679935135
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.6479342499954,127.011869707413
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.5446697303679,126.835179100171
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.4873960036367,126.927113892829
...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.4890451973362,126.500458122035
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.4815759715974,126.520156152743
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.5395361411297,126.643719243436
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.4096918414655,126.268544091817


In [35]:
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.5179681611717,127.047059839521
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.5450208674213,127.13679935135
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.6479342499954,127.011869707413
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.5446697303679,126.835179100171
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.4873960036367,126.927113892829
...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.4890451973362,126.500458122035
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.4815759715974,126.520156152743
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.5395361411297,126.643719243436
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.4096918414655,126.268544091817


In [47]:
import folium

map = folium.Map(location=[37.56682, 126.97865], zoom_start=13)  # 중심 위치

for i in df.index:
    if pd.notnull(df['위도'][i]) and pd.notnull(df['경도'][i]):  # None 값인 경우 건너뜀
        folium.Marker(
            location=[df['위도'][i], df['경도'][i]],  # WGS84 좌표계: 위도(latitude), 경도(longitude)
            tooltip=df['측정소명'][i],
            popup=folium.Popup(df['주소'][i], max_width=200)
        ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">측정소 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))

map

In [53]:
df

['서울', '강남구', '학동로', '426', '강남구청', '별관', '1동']

In [185]:
df = pd.read_csv('data/측정소좌표.csv', index_col="Unnamed: 0")
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114
...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544


In [168]:
df.주소

0                              서울 강남구 학동로 426 강남구청 별관 1동
1                           서울 강동구 구천면로 42길 59 천호1동 주민센터
2                            서울 강북구 삼양로 139길 49 우이동 주민센터
3                     서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관
4                                서울 관악구 신림동길 14 신림동 주민센터
                             ...                        
520    제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상
521                        제주 제주시 이도2동 1176-1(제주시청) 제주시청
522                   제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관
523                   제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관
524                      제주특별자치도 제주시 화북일동 1098 지상 (화북일동)
Name: 주소, Length: 525, dtype: object

In [169]:
df.주소[100].split()[:3]

['경기도', '화성시', '우정읍']

In [170]:
gu_list = []
for i in range(len(df.주소)):
    gu_list.append(df.주소[i].split()[:3])
df['구'] = gu_list
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,"[서울, 강남구, 학동로]"
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,"[서울, 강동구, 구천면로]"
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,"[서울, 강북구, 삼양로]"
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,"[서울, 강서구, 강서로]"
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,"[서울, 관악구, 신림동길]"
...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,"[제주, 제주시, 연동]"
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,"[제주, 제주시, 이도2동]"
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,"[제주특별자치도, 제주시, 조천읍]"
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,"[제주특별자치도, 제주시, 한림읍]"


In [171]:
for i in range(len(df.구)):
    if ("구" not in df.구[i]):
        df['구'][i] = df.구[i][1]
    else:
        df['구'][i] = df.구[i][2]
df

C:\Users\human\AppData\Local\Temp\ipykernel_8440\212769772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['구'][i] = df.구[i][1]


,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구
...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시


In [172]:
df.to_csv("data/측정소좌표(GU구분).csv")
df

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구
...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시


In [248]:
df2 = pd.read_csv("data/측정소좌표(GU구분).csv", index_col="Unnamed: 0")
df2

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구
...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시


In [250]:
df3 = pd.read_csv("data/위도경도_최종.csv")
df3

,ID,광역시도,시군구,격자 X,격자 Y
0,서울 종로,서울특별시,종로구,60,127
1,서울 중구,서울특별시,중구,60,127
2,서울 용산,서울특별시,용산구,60,126
3,서울 성동,서울특별시,성동구,61,127
4,서울 광진,서울특별시,광진구,62,126
...,...,...,...,...,...
245,화천,강원특별자치도,화천군,72,139
246,양구,강원특별자치도,양구군,77,139
247,인제,강원특별자치도,인제군,80,138
248,고성(경남),강원특별자치도,고성군,85,145


In [251]:
df3 = df3.시군구
gu = df3.to_list()
gu

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구',
 '중구',
 '서구',
 '동구',
 '영도구',
 '부산진구',
 '동래구',
 '남구',
 '북구',
 '해운대구',
 '사하구',
 '금정구',
 '강서구',
 '연제구',
 '수영구',
 '사상구',
 '기장군',
 '중구',
 '동구',
 '서구',
 '남구',
 '북구',
 '수성구',
 '달서구',
 '달성군',
 '중구',
 '동구',
 '미추홀구',
 '연수구',
 '남동구',
 '부평구',
 '계양구',
 '서구',
 '강화군',
 '옹진군',
 '동구',
 '서구',
 '남구',
 '북구',
 '광산구',
 '동구',
 '중구',
 '서구',
 '유성구',
 '대덕구',
 '중구',
 '남구',
 '동구',
 '북구',
 '울주군',
 '세종특별자치시',
 '수원시장안구',
 '수원시권선구',
 '수원시팔달구',
 '수원시영통구',
 '성남시수정구',
 '성남시중원구',
 '성남시분당구',
 '의정부시',
 '안양시만안구',
 '안양시동안구',
 '부천시',
 '광명시',
 '평택시',
 '동두천시',
 '안산시상록구',
 '안산시단원구',
 '고양시덕양구',
 '고양시일산동구',
 '고양시일산서구',
 '과천시',
 '구리시',
 '남양주시',
 '오산시',
 '시흥시',
 '군포시',
 '의왕시',
 '하남시',
 '용인시처인구',
 '용인시기흥구',
 '용인시수지구',
 '파주시',
 '이천시',
 '안성시',
 '김포시',
 '화성시',
 '광주시',
 '양주시',
 '포천시',
 '여주시',
 '연천군',
 '가평군',
 '양평군',
 '청주시상당구',
 '청주시서원구',
 

In [252]:
juso = df.주소.to_list()

In [253]:
juso

['서울 강남구 학동로 426 강남구청 별관 1동',
 '서울 강동구 구천면로 42길 59 천호1동 주민센터',
 '서울 강북구 삼양로 139길 49 우이동 주민센터',
 '서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관',
 '서울 관악구 신림동길 14 신림동 주민센터',
 '서울특별시 광진구 광나루로 571 구의 아리수정수센터',
 '서울 구로구 가마산로 27길 45 구로고등학교',
 '서울 금천구 금하로21길 20 시흥5동 주민센터',
 '서울 노원구 상계로 118 상계2동 주민센터 (23길 17 노원구 원터행복발전소)',
 '서울 도봉구 시루봉로2길 34 쌍문동청소년문화의집',
 '서울 동대문구 천호대로13길 43 용두초등학교',
 '서울 동작구 사당로16아길 6 사당4동 주민센터',
 '서울 마포구 포은로 6길 10 망원1동주민센터 옥상',
 '서울 서대문구 세검정로4길 32(홍제3동 주민센터) ',
 '서울 서초구 신반포로15길 16 반포 2동 주민센터',
 '서울 성동구 뚝섬로3길 18 성수1가1동주민센터',
 '서울 성북구 삼양로2길 70 길음2동 주민센터',
 '서울 송파구 백제고분로 236 삼전동 주민센터 (삼전동)',
 '서울 양천구 중앙로52길 56 신정4동 문화센터',
 '서울특별시 영등포구 당산로 123 영등포구청 (당산동3가)',
 '서울 용산구 한남대로 136 서울특별시중부기술교육원',
 '서울 은평구 진흥로 215 (한국환경산업기술원 온실동2층 )',
 '서울 종로구 종로35가길 19 종로5,6가 동 주민센터',
 '서울 중구 덕수궁길 15 시청서소문별관 3동',
 '서울 중랑구 용마산로 369 건강가정지원센터',
 '경기도 여주시 가남읍 태평중앙1길 20 가남읍행정복지센터 옥상',
 '경기도 가평군 가평읍 석봉로 181 가평군청 의회동',
 '경기 광주시 중앙로 128 농협중앙회',
 '경기도 평택시 고덕면 고덕국제2로 111 종덕초등학교 2층 옥상',
 '경기 수원시 권선구 서부로 1600 수원시도로교통관리사업소

In [246]:
juso[0].split()

['서울', '강남구', '학동로', '426', '강남구청', '별관', '1동']

In [247]:
#  '구' 열을 초기화합니다.
df['구'] = None

 # df3는 행정동명 데이터가 들어있는 리스트.
for j in range(len(juso)):
      for i in gu:
            for ju in juso[j].split():
                  if ju in i:
                        df.at[j, '구'] = i  # 해당 행의 '구' 열에 행정동명을 추가합니다.
df

AttributeError: 'float' object has no attribute 'split'